In [ ]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # 모델 바디를 로드합니다. 
        self.roberta = RobertaModel(config, add_pooling_layer = False)
        # 토큰 분류 헤드를 준비
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.linear(config.hidden_size, config.num_labels)
        # 가중치를 로드하고 초기화 합니다. 
        self.init_weights()
        
    def forward(self, input_ids = None, attention_mask = None, token_type_ids = None, labels = None, **kwargs):
        
        # 모델 바디를 사용해 인코더 표현을 얻습니다.
        outputs = self.roberta(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, **kwargs)
        
        # 인코더 표현을 헤드에 통과시킵니다. 
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # 손실을 계산합니다. 
        loss = None
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # 모델 출력 객체를 반환합니다. 
        return TokenClassifierOutput(loss = loss, 
                                     logits = logits, 
                                     hidden_states = outputs.hidden_states, 
                                     attentions = outputs.attentions)    

# 08. 클래스

## 8.1 클래스 소개

**생산 공정과 클래스**

소프트웨어 설계 분야에서도 붕어빵 틀이나 자동차의 프레임 같은 개념이 도입 --> 클래스(class)

객체 지향 프로그래밍은 프로그램을 객체라는 기본 단위로 나누고 이들의 상호 작용을 서술하는 방식

* 객체 지향 프로그래밍으로 구현

먼저 틀이나 설계도 역할을 하는 SuperMario 클래스를 정의

설계도로부터 생성된 것을 객체 -> 함수, 데이터  ~~~~ 

붕어빵 틀이 클래스, 붕어빵 틀로 생성된 것을 객체한다. 



In [2]:
class SuperMario:
    def __init__(self):
        self.xpos = 10
        self.ypos = 0
        
    def move(self, xoffset, yoffset):
        self.xpos += xoffset
        self.ypos += yoffset
             
mario_1p = SuperMario()
mario_2p = SuperMario()
             
mario_1p.move(20, 0) # x축의 20만큼 움직임
mario_2p.move(30, 0) # x축의 30만큼 움직임
             
print("mario 1p", mario_1p.xpos, mario_1p.ypos)

print("mario 2p", mario_2p.xpos, mario_2p.ypos)
               

mario 1p 30 0
mario 2p 40 0


**클래스와 객체**

클래스 --> 만들어진 실체: 객체 

객체는 영어 단어 발음 그대로 오브젝트, 인스턴스라고도 불리우니 

객체, 오브젝트, 인스턴스는 모두 같은 의미를 가짐

**파이썬 클래스**

- 절차 지향 프로그래밍은 데이터와 데이터를 처리하는 함수가 파이썬 파일 안에 위치

- 객체 지향 프로그래밍에서는 데이터와 이 데이터를 처리하는 함수를 클래스라는 공간 안으로 넣어주는 개념

- 함수와 데이터가 파일 안에서 위치하는 것이 아니라 파일 안에 클래스라는 공간을 하나 더 만들고 그 공간 안으로 데이터와 이 데이터를 처리하는 함수를 같이 넣어줌

클래스 공간 안으로 데이터와 데이터를 처리하는 함수를 같이 넣어두고 이 틀 또는 설계도를 사용하는 것이 바로 객체 지향 프로그래밍

https://hwiyong.tistory.com/392

## 8.2 클래스 정의

클래스를 정의하면 메모리에 클래스를 위한 공간이 할당됨

  - 그 공간 안으로 변수나 함수를 넣을 수가 있음

In [3]:
class 붕어빵틀:
    pass

내빵 = 붕어빵틀()

In [4]:
class 붕어빵틀:
    pass

내빵 = 붕어빵틀()
너빵 = 붕어빵틀()

객체 공간에 직접 데이터를 넣어보는 것을 코딩


In [5]:
class 붕어빵틀:
    pass

내빵 = 붕어빵틀()
너빵 = 붕어빵틀()
내빵.앙꼬 = "딸기맛"
너빵.앙꼬 = "초코맛"

In [6]:
내빵.앙꼬

'딸기맛'

내빵이라는 변수느 객체 공간을 바인딩 --> 변수 다음에 점(.)을 찍으면 그 공간 안을 의미하고

따라서 내빵.앙꼬 = "딸기맛"은 내빵이 가리키는 메모리 공간 안에 변수인 앙꼬와 값이 딸기맛이 저장됨

이때 딕셔너리가 사용되어 {"앙꼬":"딸기맛"}으로 저장됨

In [7]:
print(내빵.앙꼬)
print(너빵.앙꼬)

딸기맛
초코맛


## 8.3 클래스와 메서드

클래스 안에 정의된 함수를 특별히 메서드(method)라고 부름

  - 클래스의 동작을 표현하는데 사용됨

데이터는 클래스 공간관 객체 공간에 모두 저장될 수 있는데 일반적으로 객체 공간에 저장됨

  - 왜냐하면 각 객체는 동일한 클래스로부터 생성되었지만 데이터는 서로 달라야하기 때문에

이전 절에서 붕어빵 클래스로부터 생성된 객체 공간에 앙꼬를 넣는 행위를 함수로 만들고 클래스 안에 정의해주면 이를 메서드라고 부름

In [8]:
class 붕어빵틀:
    def 앙꼬넣기(어떤빵, 넣을앙꼬):
        어떤빵.앙꼬 = 넣을앙꼬

내빵 = 붕어빵틀()

**메서드 호출하기**

함수를 정의했다면 함수이름()과 같은 형태로 함수를 호출할 수 있음

메서드 역시 클래스 공간안에 위치하는 함수이므로 동일하게 호출이 가능

다만 클래스 안에 위치하기 때문에 먼저 클래스이름을 통해 클래스 공간에 접근

클래스 공간에 접근하려면 클래스 이름에 점(.)을 찍어주면 됨

In [12]:
class 붕어빵틀:
    def 앙꼬넣기(어떤빵, 넣을앙꼬):
        어떤빵.앙꼬 = 넣을앙꼬

내빵 = 붕어빵틀()

# 메서드 호출
붕어빵틀.앙꼬넣기(내빵, "딸기슈크림맛")

In [13]:
내빵.앙꼬

'딸기슈크림맛'

## 8.4 파이썬 클래스 self 이해하기

In [14]:
class 붕어빵틀:
    def 앙꼬넣기(어떤빵, 넣을앙꼬):
        어떤빵.앙꼬 = 넣을앙꼬 # 넣을앙꼬 함수의 파라미터, 앙꼬는 객체 내의 변수

In [15]:
class 계좌:
    def 개설(누구계좌, 이름, 잔고):
        누구계좌.이름 = 이름 # 오른쪽 이름은 함수의 파라미터, 왼쪽은 객체 내의 변수
        누구계좌.잔고 = 잔과

파이썬에서는 관례적으로 클래스 내의 함수인 메서드의 첫 번째 파라미터의 이름을 통일해서 사용함 

그 이름이 바로 self임

붕어빵 클래스에서 어떤 빵이라는 변수를 self 치환

계좌 클래스에서 누구 계좌라는 변수도 self로 치환


In [16]:
class 붕어빵틀:
    def 앙꼬넣기(self, 넣을앙꼬):
        self.앙꼬 = 넣을앙꼬 # 넣을앙꼬 함수의 파라미터, 앙꼬는 객체 내의 변수

In [17]:
class 계좌:
    def 개설(self, 이름, 잔고):
        self.이름 = 이름 # 오른쪽 이름은 함수의 파라미터, 왼쪽은 객체 내의 변수
        self.잔고 = 잔과

이것이 바로 파이썬의 self 정체다. 정리해보면 self는 메서드가 호출될 때 해당 메서드가 참조할 객체를 바인딩하는 변수

그래서 클래스 안에 정의된 모든 메서드의 첫 번재 인자로 self를 사용한다. 

## 8.5 메서드 호출 방식

**메서드 호출**

앞에서 메서드의 첫 번째 인자는 self이고 self는 메서드 호출 시 어떤 객체를 바인딩한다고 했음


In [18]:
class 붕어빵틀:
    def 앙꼬넣기(self, 앙꼬):
        self.앙꼬 = 앙꼬 # 오른쪽 앙꼬는 파라미터, 왼쪽 앙꼬는 객체 내 변수

In [19]:
내빵 = 붕어빵틀()
붕어빵틀.앙꼬넣기(내빵, '딸기맛크림')

In [20]:
print(내빵.앙꼬)

딸기맛크림


붕어빵틀 클래스로부터 내빵이라는 하나의 객체를 생성했다.

그리고 내빵에 앙꼬를 넣기위해 붕어빵틀 클래스 공간에 있는 앙꼬 넣기를 호출했다. 

이때 첫 번째 인자로 내빵을 넣어주고 두 번재 인자로 "딸기맛크림"을 전달함

내빵에 저장된 앙꼬를 참조하기 위해 내빵.앙꼬를 화면에 프린트함

앞서 객체와 클래스 공간에서 점(.)은 그 안을 의미

따라서 내빵.앙꼬는 내빵이라는 변수가 바인딩하는 객체 공간안에서 앙꼬라는 변수를 찾고 이를 출력함

**메서드 호출 방식의 비교**

|방식 | 코드 예시 | 특징 |
|:----|:-----------|:------|
|클래스 공간으로부터 호출| 붕어빵틀.앙꼬넣기(내빵, '딸기맛') | - |
|객체로부터 호출 | 내빵.앙꼬넣기("딸기맛") | self 자리에 사용자가 인자를 넘겨 줄 필요가 없다.|


**메서드 호출 이해하기**

일반적으로 객체를 이용한 메서드 호출 방식을 사용한다.

내빵.앙꼬넣기("딸기맛") # self 파라미터는 자동으로 넘어옴

In [21]:
a = [1, 2, 3] # list 클래스의 객체 생성
a.append(4)   # list 클래스의 정의된 append 메서드를 a 객체에 대해서 호출
print(a)

[1, 2, 3, 4]


1. 리스트 클래스의 객체를 생성

2. 해당 객체로부터 append 메서드를 호출

3. append 메서드의 첫 번째 인자 self에는 메서드를 호출하는 객체인 a가 자동으로 전달

4. 원소를 추가하는 append 메서드는 a 객체에 원소를 추가함

In [22]:
b = "hello" # b는 문자열 클래스의 객체
b.upper()  # 문자열 클래스의 upper 메서드 호출 (b 객체에 대해서)

'HELLO'

문자열 클래스로부터 객체를 생성하고 해당 객체를 b라는 변수가 바인딩함

따라서 b라는 변수를 통해 문자열 클래스에 있는 메서드를 호출할 수 있음

이처럼 파이썬의 거의 모든 문법은 클래스를 기반으로 함.

따라서 클래스에 대한 이해도에 따라 전체적인 문법 이해도가 달라짐

## 8.6 생성자

객체를 생성한 후 해당 객체에 데이터를 넣는 것이 아니라 객체가 생성될 때 데이터를 넣어주는 것이 편리함

> 파이썬은 이를 위해 객체가 생성될 때 파이썬 인터프리터에 의해 자동으로 호출되는 특별한 메서드를 제공함. 중요한 점은 메서드를 사용자가 호출하는게 아니라 객체가 생성되면 자동으로 호출이 됨

이 메서드를 생성자라고 부르며 `__init__` 이라는 이름을 가짐

In [23]:
class 붕어빵틀:
    def __init__(self):
        print("붕어빵 구어짐")

앞서 생성자는 객체가 생성될 때 자동으로 호출됨

In [24]:
내빵 = 붕어빵틀()

붕어빵 구어짐


`붕어빵 구어짐` 출력되는 이유는 객체가 생성될 때 파이썬 인터프리터가 `__init__` 메서드를 호출했기 때문임. 

**생성자의 이름은 `__init__` 이다.**

**생성자는 객체가 생성될 때 자동으로 호출된다.**

생성자를 사용하면 객체가 생성될 때 앙꼬를 넣을 수 있음

기존의 앙꼬넣기 메서드의 이름을 __init__ 으로 변경함. 

  - 객체가 생성될 때 이제 자동으로 앙꼬를 넣는 생성자가 호출됨
  
  - 따라서 객체를 생성할 때 '딸기맛'이라는 앙꼬도 같이 전달해줘야 함

In [25]:
class 붕어빵틀:
    def __init__(self, 앙꼬):
        self.앙꼬 = 앙꼬 # 오른쪽 앙꼬는 파라미터, 왼쪽 앙꼬는 객체 내의 변수
        
내빵 = 붕어빵틀("딸기맛")

만약에 `__init__`의 인자 중 self를 제외한 인자를 넣지 않고 클래스를 이용해 객체를 생성하면 에러가 발생함

In [26]:
내빵 = 붕어빵틀()

TypeError: 붕어빵틀.__init__() missing 1 required positional argument: '앙꼬'

## 8.7 클래스 상속(inheritance)

**클래스 상속**

클래스를 사용함으로써 여러가지 이점이 있는데 이번 절에서 배울 상속도 그 중 하나임

기존의 클래스 기능을 그대로 가져온 후 기능을 추가할 수 있는 것이 바로 상속(inheritance)

간단한 예제를 통해서 상속 기능을 이해해 봄

In [1]:
class Parent:
    def sing(self):
        print("sing a song")
        
father = Parent()
father.sing()

sing a song


In [2]:
class LuckyChild(Parent):
    pass

luckyboy = LuckyChild()

In [3]:
luckyboy.sing()

sing a song


In [4]:
type(luckyboy)

__main__.LuckyChild

luckyboy.sing() 이라는 구문을 살펴보면 sing 메서드를 호출.

이때 sing 메서드는 먼저 luckyboy 객체에서 찾음.

해당 객체에 없다면 클래스 공간인 LuckyChild에서 sing을 찾음

그래도 없다면 상속받은 클래스 공간인 Parent에서 sing을 찾게 되는데,

Parent에서 sing 메서드가 존재하므로 Parent에서 sing을 호출함

**속성 참조**

In [1]:
class Parent:
    def sing(self):
        print("sing a song")
        
class LuckyChild(Parent):
    def dance(self):
        print("shuffle dance")
        
luckyboy = LuckyChild()
luckyboy.sing()
luckyboy.dance()

sing a song
shuffle dance


In [2]:
my_parent = Parent()

In [3]:
my_parent.dance()

AttributeError: 'Parent' object has no attribute 'dance'